In [3]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve
import tarfile
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, datasets, transforms
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
import time
from tqdm.auto import tqdm
cudnn.benchmark = True

In [4]:
input_size = 224

In [12]:
mobilenet_v2 = models.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [26]:
to_tensor =  transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor()])

In [27]:
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [6]:
dataset_url = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz'
db = download_url(dataset_url,'.')

  0%|          | 0/98752094 [00:00<?, ?it/s]

In [7]:
# Extract from archive
with tarfile.open('./imagenette-160.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
    
# Look into the data directory
data_dir = './data/imagenette-160'
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'val']
['n02102040', 'n03394916', 'n03425413', 'n03888257', 'n02979186', 'n03000684', 'n03417042', 'n03028079', 'n03445777', 'n01440764']


In [28]:
# PyTorch datasets
train_ds = ImageFolder('data/imagenette-160/train', to_tensor)
valid_ds = ImageFolder('data/imagenette-160/val', to_tensor)

In [11]:
batch_size = 64

In [29]:
imagenette_trainloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size,
                                          shuffle=True)#, num_workers=2)
imagenette_testloader = torch.utils.data.DataLoader(valid_ds, batch_size=batch_size,
                                         shuffle=False)#, num_workers=2)
imagenette_loaders = {'train': imagenette_trainloader, 'val': imagenette_testloader}

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [13]:
model = mobilenet_v2.to(device)
train_optimizer = torch.optim.SGD(mobilenet_v2.parameters(), lr=0.001, momentum=0.9)

In [9]:
criterion = nn.CrossEntropyLoss()

In [14]:
def train_model(model, dataloaders, criterion, optimizer,
                phases, num_epochs=3):
  
    start_time = time.time()

    acc_history = {k: list() for k in phases}
    loss_history = {k: list() for k in phases}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                # print("labels: ", labels)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double()
            epoch_acc /= len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss,
                                                       epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60,
                                                        time_elapsed % 60))

    return model, acc_history

In [30]:
train_model(model, imagenette_loaders, criterion, train_optimizer,
            phases=['train', 'val'], num_epochs=2)

Epoch 0/1
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.8456 Acc: 0.8662


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.1531 Acc: 0.9600

Epoch 1/1
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.0944 Acc: 0.9717


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.1182 Acc: 0.9680

Training complete in 2m 45s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [16]:
mobilenet_v2_raw = models.mobilenet_v2()

In [17]:
model_raw = mobilenet_v2_raw.to(device)
train_optimizer_raw = torch.optim.SGD(mobilenet_v2_raw.parameters(), lr=0.001, momentum=0.9)

In [18]:
train_model(model_raw, imagenette_loaders, criterion, train_optimizer_raw,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7004 Acc: 0.1912


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.1284 Acc: 0.2780

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.8398 Acc: 0.7255


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.8124 Acc: 0.7260

Training complete in 13m 27s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [20]:
# PyTorch datasets
train_ds_normalize = ImageFolder('data/imagenette-160/train', normalize)
valid_ds_normalize = ImageFolder('data/imagenette-160/val', normalize)

In [21]:
imagenette_trainloader_normalize = torch.utils.data.DataLoader(train_ds_normalize, batch_size=batch_size,
                                          shuffle=True)
imagenette_testloader_normalize = torch.utils.data.DataLoader(valid_ds_normalize, batch_size=batch_size,
                                         shuffle=False)
imagenette_loaders_normalize = {'train': imagenette_trainloader_normalize, 'val': imagenette_testloader_normalize}


In [22]:
mobilenet_v2_normalize = models.mobilenet_v2()

In [23]:
model_normalize = mobilenet_v2_normalize.to(device)
train_optimizer_normalize = torch.optim.SGD(mobilenet_v2_normalize.parameters(), lr=0.001, momentum=0.9)

In [24]:
train_model(model_normalize, imagenette_loaders_normalize, criterion, train_optimizer_normalize,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7394 Acc: 0.1759


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.0594 Acc: 0.2320

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.8072 Acc: 0.7371


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.8947 Acc: 0.7180

Training complete in 14m 11s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [26]:
gaussian_blur = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.GaussianBlur(kernel_size=3)])

In [27]:
# PyTorch datasets
train_ds_gaussian_blur = ImageFolder('data/imagenette-160/train', gaussian_blur)
valid_ds_gaussian_blur = ImageFolder('data/imagenette-160/val', gaussian_blur)

In [28]:
imagenette_trainloader_gaussian_blur = torch.utils.data.DataLoader(train_ds_gaussian_blur, batch_size=batch_size, shuffle=True)
imagenette_testloader_gaussian_blur = torch.utils.data.DataLoader(valid_ds_gaussian_blur, batch_size=batch_size, shuffle=False)
imagenette_loaders_gaussian_blur = {'train': imagenette_trainloader_gaussian_blur, 'val': imagenette_testloader_gaussian_blur}

In [29]:
mobilenet_v2_gaussian_blur = models.mobilenet_v2()

In [30]:
model_gaussian_blur = mobilenet_v2_gaussian_blur.to(device)
train_optimizer_gaussian_blur = torch.optim.SGD(mobilenet_v2_gaussian_blur.parameters(), lr=0.001, momentum=0.9)

In [31]:
train_model(model_gaussian_blur, imagenette_loaders_gaussian_blur, criterion, train_optimizer_gaussian_blur,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7467 Acc: 0.1709


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.0476 Acc: 0.2380

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.8814 Acc: 0.7137


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.8481 Acc: 0.7360

Training complete in 17m 12s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [32]:
random_crop = transforms.Compose([
    transforms.Resize(input_size),
    transforms.RandomCrop(input_size),
    transforms.ToTensor(),])

In [33]:
# PyTorch datasets
train_ds_random_crop = ImageFolder('data/imagenette-160/train', random_crop)
valid_ds_random_crop = ImageFolder('data/imagenette-160/val', random_crop)

In [34]:
imagenette_trainloader_random_crop = torch.utils.data.DataLoader(train_ds_random_crop, batch_size=batch_size, shuffle=True)
imagenette_testloader_random_crop = torch.utils.data.DataLoader(valid_ds_random_crop, batch_size=batch_size, shuffle=False)
imagenette_loaders_random_crop = {'train': imagenette_trainloader_random_crop, 'val': imagenette_testloader_random_crop}

In [35]:
mobilenet_v2_random_crop = models.mobilenet_v2()

In [36]:
model_random_crop = mobilenet_v2_random_crop.to(device)
train_optimizer_random_crop = torch.optim.SGD(mobilenet_v2_random_crop.parameters(), lr=0.001, momentum=0.9)

In [37]:
train_model(model_random_crop, imagenette_loaders_random_crop, criterion, train_optimizer_random_crop,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7143 Acc: 0.1914


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.1733 Acc: 0.1880

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.9327 Acc: 0.6968


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.9141 Acc: 0.6940

Training complete in 13m 26s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [39]:
random_horizontal_flip = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip()])

In [48]:
# PyTorch datasets
train_ds_horizontal_flip = ImageFolder('data/imagenette-160/train', random_horizontal_flip)
valid_ds_horizontal_flip = ImageFolder('data/imagenette-160/val', random_horizontal_flip)

In [49]:
imagenette_trainloader_horizontal_flip = torch.utils.data.DataLoader(train_ds_horizontal_flip, batch_size=batch_size, shuffle=True)
imagenette_testloader_horizontal_flip = torch.utils.data.DataLoader(valid_ds_horizontal_flip, batch_size=batch_size, shuffle=False)
imagenette_loaders_horizontal_flip = {'train': imagenette_trainloader_horizontal_flip, 'val': imagenette_testloader_horizontal_flip}

In [50]:
mobilenet_v2_horizontal_flip = models.mobilenet_v2()

In [51]:
model_horizontal_flip = mobilenet_v2_horizontal_flip.to(device)
train_optimizer_horizontal_flip = torch.optim.SGD(mobilenet_v2_horizontal_flip.parameters(), lr=0.001, momentum=0.9)

In [52]:
train_model(model_horizontal_flip, imagenette_loaders_horizontal_flip, criterion, train_optimizer_horizontal_flip,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7118 Acc: 0.1938


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.0520 Acc: 0.2540

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.8898 Acc: 0.7119


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.8563 Acc: 0.7380

Training complete in 13m 43s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [63]:
random_several_transforms = transforms.Compose([
    transforms.Resize(input_size),
    transforms.RandomRotation(degrees=15),
    transforms.RandomCrop(input_size),        
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)) ])

In [64]:
# PyTorch datasets
train_ds_several_transforms = ImageFolder('data/imagenette-160/train', random_several_transforms)
valid_ds_several_transforms = ImageFolder('data/imagenette-160/val', random_several_transforms)

In [65]:
imagenette_trainloader_several_transforms = torch.utils.data.DataLoader(train_ds_several_transforms, batch_size=batch_size, shuffle=True)
imagenette_testloader_several_transforms = torch.utils.data.DataLoader(valid_ds_several_transforms, batch_size=batch_size, shuffle=False)
imagenette_loaders_several_transforms = {'train': imagenette_trainloader_several_transforms, 'val': imagenette_testloader_several_transforms}

In [66]:
mobilenet_v2_several_transforms = models.mobilenet_v2()

In [67]:
model_several_transforms = mobilenet_v2_several_transforms.to(device)
train_optimizer_several_transforms = torch.optim.SGD(mobilenet_v2_several_transforms.parameters(), lr=0.001, momentum=0.9)

In [68]:
train_model(model_several_transforms, imagenette_loaders_several_transforms, criterion, train_optimizer_several_transforms,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7432 Acc: 0.1757


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.0534 Acc: 0.2480

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.9842 Acc: 0.6759


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.9165 Acc: 0.7020

Training complete in 14m 55s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

In [18]:
random_horizontal_flip_blur = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur(kernel_size=3)])

In [19]:
# PyTorch datasets
train_ds_horizontal_flip_blur = ImageFolder('data/imagenette-160/train', random_horizontal_flip_blur)
valid_ds_horizontal_flip_blur = ImageFolder('data/imagenette-160/val', random_horizontal_flip_blur)

In [20]:
imagenette_trainloader_horizontal_flip_blur = torch.utils.data.DataLoader(train_ds_horizontal_flip_blur, batch_size=batch_size, shuffle=True)
imagenette_testloader_horizontal_flip_blur = torch.utils.data.DataLoader(valid_ds_horizontal_flip_blur, batch_size=batch_size, shuffle=False)
imagenette_loaders_horizontal_flip_blur = {'train': imagenette_trainloader_horizontal_flip_blur, 'val': imagenette_testloader_horizontal_flip_blur}

In [22]:
mobilenet_v2_horizontal_flip_blur = models.mobilenet_v2()

In [23]:
model_horizontal_flip_blur = mobilenet_v2_horizontal_flip_blur.to(device)
train_optimizer_horizontal_flip_blur = torch.optim.SGD(mobilenet_v2_horizontal_flip_blur.parameters(), lr=0.001, momentum=0.9)

In [24]:
train_model(model_horizontal_flip_blur, imagenette_loaders_horizontal_flip_blur, criterion, train_optimizer_horizontal_flip_blur,
            phases=['train', 'val'], num_epochs=10)

Epoch 0/9
----------


  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 2.7209 Acc: 0.1828


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 2.0487 Acc: 0.2540

Epoch 1/9
----------


...

  0%|          | 0/202 [00:00<?, ?it/s]

train Loss: 0.8768 Acc: 0.7151


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.8502 Acc: 0.7360

Training complete in 17m 45s


(MobileNetV2(
   (features): Sequential(
     (0): ConvNormActivation(
       (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU6(inplace=True)
     )
     (1): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
           (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): ReLU6(inplace=True)
         )
         (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
     )
     (2): InvertedResidual(
       (conv): Sequential(
         (0): ConvNormActivation(
           (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
           (1): Bat

| Strategy Name | Strategy Description | Result after 10 epochs | Best Result|
|---------------|:---------------------|:-----------------------|:-----------|
|  None         |CenterCrop & ToTensor| **0.7260** |0.7260
| Normalize     |CenterCrop & ToTensor &Normalize|0.7180  |0.7360 |
|Gaussian blur  |CenterCrop & ToTensor & Gaussian Blur| 0.7360|0.7360|
|Random Crop  | RandomCrop & ToTensor |0.6940 | 0.6940|
|Random horizontal flip | CenterCrop & ToTensor & RandomHorizontalFlip | **0.7380** | **0.7380**|
|several transforms|Resize & RandomRotation(degrees=15) & RandomCrop & ToTensor() & Normalize | 0.7020 | 0.7020|
|several transforms| CenterCrop & ToTensor & RandomHorizontalFlip() &GaussianBlur(kernel_size=3)|0.7360|0.7360|